```
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!! This notebook is a work-in-progress !!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
```

In [1]:
# !pip3 install --quiet --upgrade --user hpecp

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('/Users/christophersnow/Code/hpecp-client/'))

You **MUST** restart your kernel (MENU: Kernel -> Restart) to use the installed packages.

In [3]:
from hpecp import ContainerPlatformClient

hpeclient = ContainerPlatformClient(username='admin', 
                                password='admin123', 
                                api_host='127.0.0.1', 
                                api_port=8080,
                                use_ssl=True,
                                verify_ssl='/certs/hpecp-ca-cert.pem')

hpeclient.create_session()

In [4]:
print(hpeclient.k8s_worker.get_k8shosts().tabulate())

+-----------+------------+------------------------------------------+------------+--------------------------+
| worker_id |   status   |                 hostname                 |   ipaddr   |           href           |
+-----------+------------+------------------------------------------+------------+--------------------------+
|     3     | configured | ip-10-1-0-145.eu-west-2.compute.internal | 10.1.0.145 | /api/v2/worker/k8shost/3 |
|     4     | configured | ip-10-1-0-50.eu-west-2.compute.internal  | 10.1.0.50  | /api/v2/worker/k8shost/4 |
|     5     | configured | ip-10-1-0-181.eu-west-2.compute.internal | 10.1.0.181 | /api/v2/worker/k8shost/5 |
|     6     | configured | ip-10-1-0-118.eu-west-2.compute.internal | 10.1.0.118 | /api/v2/worker/k8shost/6 |
+-----------+------------+------------------------------------------+------------+--------------------------+


In [5]:
from hpecp import APIException
from hpecp.k8s_cluster import K8sClusterHostConfig

try:
    k8s_cluster_id = hpeclient.k8s_cluster.create(
                                        name = 'Spark Operator', 
                                        description = 'Spark Operator Demo', 
                                        k8s_version = '1.17.0', 
                                        k8shosts_config = [
                                                        K8sClusterHostConfig('/api/v2/worker/k8shost/3', 'master'),
                                                        K8sClusterHostConfig('/api/v2/worker/k8shost/4', 'worker'),
                                                        K8sClusterHostConfig('/api/v2/worker/k8shost/5', 'worker'),
                                                        K8sClusterHostConfig('/api/v2/worker/k8shost/6', 'worker')
                                                    ]
                                    )
    print('Creating cluster id: ' + k8s_cluster_id)
except APIException as e:
    print(e.message)

2020-05-02 22:31:08,940 - ContainerPlatformClient - ERROR - k8s_cluster/create : post https://127.0.0.1:8080/api/v2/k8scluster Request: {"label": {"name": "Spark Operator", "description": "Spark Operator Demo"}, "pod_network_range": "10.192.0.0/12", "service_network_range": "10.96.0.0/12", "pod_dns_domain": "cluster.local", "persistent_storage": {"local": false, "nimble_csi": false}, "k8shosts_config": [{"node": "/api/v2/worker/k8shost/3", "role": "master"}, {"node": "/api/v2/worker/k8shost/4", "role": "worker"}, {"node": "/api/v2/worker/k8shost/5", "role": "worker"}, {"node": "/api/v2/worker/k8shost/6", "role": "worker"}], "k8s_version": "1.17.0"}
Traceback (most recent call last):
  File "/Users/christophersnow/Code/hpecp-client/hpecp/client.py", line 154, in _request
    response.raise_for_status()
  File "/Users/christophersnow/Code/bluedata-demo-env-aws-terraform/venv/lib/python3.7/site-packages/requests/models.py", line 941, in raise_for_status
    raise HTTPError(http_error_msg,

403 Forbidden: 
/api/v2/worker/k8shost/3.node: host is part of an existing k8s cluster
/api/v2/worker/k8shost/4.node: host is part of an existing k8s cluster
/api/v2/worker/k8shost/5.node: host is part of an existing k8s cluster
/api/v2/worker/k8shost/6.node: host is part of an existing k8s cluster



In [6]:
print(hpeclient.k8s_cluster.list().tabulate())

+-----------------------+----------------+---------------------+-------------+--------------------+----------------------+--------------+--------+
|          id           |      name      |     description     | k8s_version | created_by_user_id | created_by_user_name | created_time | status |
+-----------------------+----------------+---------------------+-------------+--------------------+----------------------+--------------+--------+
| /api/v2/k8scluster/61 | Spark Operator | Spark Operator Demo |   1.17.0    |   /api/v1/user/5   |        admin         |  1588410471  | ready  |
+-----------------------+----------------+---------------------+-------------+--------------------+----------------------+--------------+--------+


In [7]:
from hpecp.k8s_cluster import K8sClusterStatus

hpeclient.k8s_cluster.wait_for_status(k8scluster_id='/api/v2/k8scluster/61', timeout_secs=600, status=[ K8sClusterStatus.ready ])

True

In [8]:
from hpecp import APIException

try:
    hpeclient.tenant.create(
                        name='tenant2', 
                        description='tenant demo', 
                        tenant_type='k8s', 
                        k8s_cluster='/api/v2/k8scluster/61'
                        )
except APIException as e:
    print(e.message)

2020-05-02 22:31:09,141 - ContainerPlatformClient - WARNING - !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2020-05-02 22:31:09,143 - ContainerPlatformClient - WARNING - !!!! The method `tenant.create()` is experimental !!!!
2020-05-02 22:31:09,145 - ContainerPlatformClient - WARNING - !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2020-05-02 22:31:09,254 - ContainerPlatformClient - ERROR - tenant/create : post https://127.0.0.1:8080/api/v1/tenant Request: {"label": {"name": "tenant2", "description": "tenant demo"}, "tenant_type": "k8s", "member_key_available": "all_admins", "k8s_cluster": "/api/v2/k8scluster/61"}
Traceback (most recent call last):
  File "/Users/christophersnow/Code/hpecp-client/hpecp/client.py", line 154, in _request
    response.raise_for_status()
  File "/Users/christophersnow/Code/bluedata-demo-env-aws-terraform/venv/lib/python3.7/site-packages/requests/models.py", line 941, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
re

403 Forbidden: tenant with the same name already exists



In [9]:
import os
os.environ["LOG_LEVEL"] = "DEBUG"

print(hpeclient.tenant.list().tabulate())

+-------------------+----------------------------------+--------------------------------------------------------+-------------+
|        id         |               name               |                      description                       | tenant_type |
+-------------------+----------------------------------+--------------------------------------------------------+-------------+
| /api/v1/tenant/1  |            Site Admin            |        Site Admin Tenant for BlueData clusters         |             |
| /api/v1/tenant/11 | K8S Cluster Admin Spark Operator | Kubernetes Cluster Admin Tenant for Kubernetes cluster |  k8s_admin  |
| /api/v1/tenant/2  |           Demo Tenant            |           Demo Tenant for BlueData Clusters            |   docker    |
| /api/v1/tenant/29 |             tenant2              |                      tenant demo                       |     k8s     |
+-------------------+----------------------------------+------------------------------------------------

In [10]:
%%sh

cd ..
./generated/ssh_controller.sh hostname -f

ip-10-1-0-219.eu-west-2.compute.internal


In [11]:
from kubernetes import client, config
import tempfile

try:
    demo_tenant = hpeclient.tenant.get('/api/v1/tenant/29')
    demo_cluster = hpeclient.k8s_cluster.get('/api/v2/k8scluster/61')
except APIException as e:
    print(e)

ns = demo_tenant.json['namespace']
    
# get the kube admin config for the first cluster 
admin_kube_config = demo_cluster.admin_kube_config

#  write it to a temporary file
with tempfile.NamedTemporaryFile(mode = "w") as fp:
    fp.write(admin_kube_config)
    fp.flush()

    # read the kube config with the kubernetes client
    config.load_kube_config(fp.name)

    # list the pods        
    v1 = client.CoreV1Api()
    
    print("\nListing pods in tenant namespace '{}' with their IPs:\n".format(ns))
    ret = v1.list_namespaced_pod(watch=False, namespace=ns)
    for i in ret.items:
        print("{:>12}   {:>20}   {}".format(i.status.pod_ip, i.metadata.namespace, i.metadata.name))
    
    print("\nListing pods in all namespaces with their IPs:\n")
    ret = v1.list_pod_for_all_namespaces(watch=False)
    for i in ret.items:
        print("{:>12}   {:>20}   {}".format(i.status.pod_ip, i.metadata.namespace, i.metadata.name))

2020-05-02 22:31:12,422 - ContainerPlatformClient - WARNING - !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2020-05-02 22:31:12,424 - ContainerPlatformClient - WARNING - !!!! The method `tenant.get()` is experimental !!!!
2020-05-02 22:31:12,425 - ContainerPlatformClient - WARNING - !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


hpecp-tenant-29-pdz85

Listing pods in tenant namespace 'hpecp-tenant-29-pdz85' with their IPs:


Listing pods in all namespaces with their IPs:

  10.192.1.7                  hpecp   hpecp-agent-8499d5798b-gmhp4
  10.1.0.181                  hpecp   hpecp-fsmount-258s6
  10.1.0.145                  hpecp   hpecp-fsmount-2cmbb
   10.1.0.50                  hpecp   hpecp-fsmount-9fqld
  10.1.0.118                  hpecp   hpecp-fsmount-lzflg
10.192.0.132                  hpecp   kubedirector-7fdbdb7775-gz2m7
   10.1.0.50            kube-system   canal-6fq8p
  10.1.0.145            kube-system   canal-8xcn4
  10.1.0.118            kube-system   canal-fkrq5
  10.1.0.181            kube-system   canal-z2k9s
10.192.0.133            kube-system   coredns-6955765f44-6vhrf
10.192.1.134            kube-system   coredns-6955765f44-9mtcb
  10.1.0.145            kube-system   etcd-ip-10-1-0-145.eu-west-2.compute.internal
  10.1.0.145            kube-system   kube-apiserver-ip-10-1-0-145.eu-west-2.